# Estimating Free Parking

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "free_parking"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/free_parking/free_parking_coefficients.csv
loading spec from output-est-mode/estimation_data_bundle/free_parking/free_parking_SPEC.csv
loading from output-est-mode/estimation_data_bundle/free_parking/free_parking_values_combined.parquet


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_asc_san_francisco,-2.6403,F
coef_asc_santa_clara,0.2118,F
coef_asc_alameda,-0.1092,F
coef_income_very_high,0.2300,F
coef_income_high,0.2300,F
coef_hh_size_4_up,0.2530,F
coef_more_autos_than_workers,0.2310,F
coef_fewer_autos_than_workers,-1.4790,F


#### Utility specification

In [5]:
data.spec

,Label,Description,Expression,free,pay
0,util_asc_san_francisco,NaN,@df.workplace_county_id == ID_SAN_FRANCISCO,coef_asc_san_francisco,0.0
1,util_asc_santa_clara,NaN,@df.workplace_county_id == ID_SANTA_CLARA,coef_asc_santa_clara,0.0
2,util_asc_alameda,NaN,@df.workplace_county_id == ID_ALAMEDA,coef_asc_alameda,0.0
3,util_income_very_high,Very high income household dummy,@df.income>=100000,coef_income_very_high,0.0
4,util_income_high,High income housheold dummy,@(df.income>=60000) & (df.income<100000),coef_income_high,0.0
5,util_hh_size_4_up,Household size is greater than 3 dummy,@df.hhsize>3,coef_hh_size_4_up,0.0
6,util_more_autos_than_workers,More automobiles than workers dummy,@df.auto_ownership>df.num_workers,coef_more_autos_than_workers,0.0
7,util_fewer_autos_than_workers,Fewer automobiles than workers dummy,@df.auto_ownership<df.num_workers,coef_fewer_autos_than_workers,0.0


### Chooser data

In [6]:
data.chooser_data

,person_id,model_choice,override_choice,util_asc_san_francisco,util_asc_santa_clara,util_asc_alameda,util_income_very_high,util_income_high,util_hh_size_4_up,util_more_autos_than_workers,...,auOpRetail,auOpTotal,trPkRetail,trPkTotal,trOpRetail,trOpTotal,nmRetail,nmTotal,workplace_county_id,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
72355,72355,False,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,9.915345,12.430580,6.550726,9.119016,6.446184,9.035333,5.256966,6.831275,1,2
72384,72384,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.971763,12.488100,6.384615,8.910521,6.292135,8.845287,5.447277,7.062559,1,2
72407,72407,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.971763,12.488100,6.384615,8.910521,6.292135,8.845287,5.447277,7.062559,1,2
72459,72459,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.086939,12.619691,6.787018,9.400117,6.687820,9.307625,5.877288,7.691985,1,2
72529,72529,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.263491,12.913804,6.830450,9.558905,6.611336,9.308290,6.696709,8.925142,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847736,7539071,False,True,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,9.875413,12.586445,4.800090,8.088658,4.581187,7.858361,5.697172,9.212853,4,1
2847868,7539203,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.121001,11.606573,0.000000,0.000000,0.000000,0.000000,1.324505,4.959246,5,2
2847882,7539217,True,True,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,9.121001,11.606573,0.000000,0.000000,0.000000,0.000000,1.324505,4.959246,4,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.estimate()

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0,1
coef_asc_alameda,-0.141547,-0.141547,-0.1092,-50.0,50.0,0.0,0
coef_asc_san_francisco,-2.630212,-2.630212,-2.6403,-50.0,50.0,0.0,0
coef_asc_santa_clara,0.188205,0.188205,0.2118,-50.0,50.0,0.0,0
coef_fewer_autos_than_workers,-1.407827,-1.407827,-1.4790,-50.0,50.0,0.0,0
coef_hh_size_4_up,0.264779,0.264779,0.2530,-50.0,50.0,0.0,0
coef_income_high,0.217818,0.217818,0.2300,-50.0,50.0,0.0,0
coef_income_very_high,0.235389,0.235389,0.2300,-50.0,50.0,0.0,0
coef_more_autos_than_workers,0.225131,0.225131,0.2310,-50.0,50.0,0.0,0


┣                x: 0.0                              0.000000
┃                   coef_asc_alameda                -0.141547
┃                   coef_asc_san_francisco          -2.630212
┃                   coef_asc_santa_clara             0.188205
┃                   coef_fewer_autos_than_workers   -1.407827
┃                   coef_hh_size_4_up                0.264779
┃                   coef_income_high                 0.217818
┃                   coef_income_very_high            0.235389
┃                   coef_more_autos_than_workers     0.225131
┃                   dtype: float64
┣          logloss: 0.5636931088124669
┣        d_logloss: 0.0                              0.000000
┃                   coef_asc_alameda                 0.000061
┃                   coef_asc_san_francisco           0.000074
┃                   coef_asc_santa_clara             0.000008
┃                   coef_fewer_autos_than_workers    0.000150
┃                   coef_hh_size_4_up               -0.000075
┃                   coef_income_high                 0.000317
┃                   coef_income_very_high           -0.000092
┃                   coef_more_autos_than_workers     0.000781
┃                   dtype: float64
┣              nit: 7
┣             nfev: 7
┣             njev: 7
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(microseconds=92472)
┣           method: 'slsqp'
┣          n_cases: 28281
┣ iteration_number: 7
┣          loglike: -15941.804810325377

### Estimated coefficients

In [8]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
0.0,0.00,0.00,NA,,0.00,fixed value
coef_asc_alameda,-0.142,0.0310,-4.56,***,0.00,
coef_asc_san_francisco,-2.63,0.0591,-44.52,***,0.00,
coef_asc_santa_clara,0.188,0.0294,6.40,***,0.00,
coef_fewer_autos_than_workers,-1.41,0.0366,-38.48,***,0.00,
coef_hh_size_4_up,0.265,0.0278,9.51,***,0.00,
coef_income_high,0.218,0.0307,7.09,***,0.00,
coef_income_very_high,0.235,0.0282,8.35,***,0.00,
coef_more_autos_than_workers,0.225,0.0294,7.65,***,0.00,


# Output Estimation Results

In [9]:
from activitysim.estimation.larch import update_coefficients

result_dir = data.edb_directory / "estimated"
update_coefficients(
    model,
    data,
    result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [10]:
model.to_xlsx(
    result_dir / f"{modelname}_model_estimation.xlsx",
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [11]:
pd.read_csv(result_dir / f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_asc_san_francisco,-2.630212,F
1,coef_asc_santa_clara,0.188205,F
2,coef_asc_alameda,-0.141547,F
3,coef_income_very_high,0.235389,F
4,coef_income_high,0.217818,F
5,coef_hh_size_4_up,0.264779,F
6,coef_more_autos_than_workers,0.225131,F
7,coef_fewer_autos_than_workers,-1.407827,F
